In [ ]:
import os
import utils.CONSTANTS as CONST
from collections import defaultdict
from tabulate import tabulate, SEPARATING_LINE
import pandas as pd
import subprocess

class XYRun():

    def __init__(self, runs: pd.DataFrame) -> None:

        self.runs = self.scan_runs(runs)
        xy_run = runs[runs['forDB']]
        self.year_and_month = xy_run.values[0,2].strftime("%Y-%m")
    

    def __repr__(self):
        return str(self.runs)

    
    def __getitem__(self, key):
        return self.runs.get(key, "")

    
    def run_calib(self, /, offline_version, *, rerun=True) -> None:

        if offline_version is None:
            offline_version = "../bd0e9e"
        
        ids = " ".join(self.runs.values())
        r = "-r" if rerun else ""

        move = f"cd {CONST.SCAN_PATH}"
        run_calib = f"./run_Calib.py -i {ids} {r} -cfg {self.year_and_month}"
        source_offline = f"source {CONST.OFLN_PATH / offline_version / 'set_offline_env.sh'}"

        subprocess.call("; ".join([move, source_offline, run_calib]), shell=True, executable='/bin/bash')


    @staticmethod
    def scan_runs(runs: pd.DataFrame) -> dict:
        run_dict, postXY = {}, False
        for _id, info in runs.iterrows():

            if info['forDB']: 
                postXY = True
                key = "XY"
            else:
                try:
                    key = "post" if postXY else "pre"
                    if "open shutter" in info["comment"]:
                        key += "XY"
                    else: 
                        key += "DAQ"
                except TypeError: continue

            if run_dict.get(key, None) is None or not postXY:
                run_dict[key] = _id

        return run_dict


class Campaign():
    
    def __init__(self, year: int, month: int) -> None:

        runlist = CONST.SCAN_PATH / f"config/calib_runlists/calib_runs_{year}-{month:02}.list"
        
        if not os.path.isfile(runlist):
            raise FileNotFoundError(f"{runlist} not found!\nAvailable campaigns are:\n" + 
                                    "\n".join(os.listdir(CONST.SCAN_PATH / "config/calib_runlists")))

        self.data = pd.read_csv(runlist,
                                names=["id","tel","step","date","source","mA","forDB","jobfile","comment"],
                                dtype=defaultdict(lambda: "str", step="int", mA="float", forDB="bool"),
                                skipinitialspace=True,
                                parse_dates=["date"],
                                index_col=0,
                                comment="#",
                                sep=";")
        
        self.runs = self.split_runlist(self.data)


    def __getitem__(self, tel) -> XYRun:
        return self.runs[tel.upper()]


    @staticmethod
    def split_runlist(runlist: pd.DataFrame) -> dict:

        run_dict = {}
        for _, info in runlist[runlist['forDB']].iterrows():
            
            run_dict[info['tel'].upper()] = XYRun(runlist[
                (runlist['date'] == info['date'])
                & (runlist['tel'] == info['tel'])
                ])
        
        return run_dict


    def __repr__(self) -> True:
        
        data = dict(sorted(self.runs.items()))
        table_data = []

        last = "LLLMLACOHE"
        for tel, runs in data.items():
            if tel[:-1] not in last:
                table_data.append(SEPARATING_LINE)
            
            table_data.append(
                [tel,
                 runs['preDAQ'],
                 runs['preXY'],
                 runs['XY'],
                 runs['postXY'],
                 runs['postDAQ']
                 ]
            )
            
            last = tel[:-1]

        return tabulate(table_data, disable_numparse=True,
                        headers=['tel', 'preDAQ', 'preXY', 'XY', 'postXY', 'postDAQ'])

    
    def run_calib(self, /, offline_version, *, rerun=False) -> None:
        for run in self.runs.values():
            run.run_calib(rerun, offline_version)

17:31:01 (   +3.6s) [INFO   ] -- import logging
17:31:01 (  +212ms) [INFO   ] -- set MONI_PATH = PosixPath('/cr/work/filip/monit_and_sd')
17:31:01 (    +1ms) [INFO   ] -- set HIST_PATH = PosixPath('/cr/work/filip/monit_and_sd')
17:31:01 (    +1ms) [INFO   ] -- set PLOT_PATH = PosixPath('/cr/data01/filip/plots')
17:31:01 (    +1ms) [INFO   ] -- set DATA_PATH = PosixPath('/cr/data01/filip/Data')
17:31:01 (    +0ms) [INFO   ] -- set SCAN_PATH = PosixPath('/cr/data01/filip/xy-calibration')
17:31:01 (    +1ms) [INFO   ] -- set OFLN_PATH = PosixPath('/cr/data01/filip/offline/install')


In [2]:
test = Campaign(2023,10)
test['he1'].run_calib()



  ___   ___ ____    ____         ______     ___       __      
  \  \ /  / \   \  /   /        /      |   /   \     |  |     
   \  V  /   \   \/   / _____  |  ,----'  /  ^  \    |  |     
    >   <     \_    _/ |_____| |  |      /  /_\  \   |  |     
   /  .  \      |  |           |  `----./  _____  \  |  `----.
  /__/ \__\     |__|            \______/__/     \__\ |_______|


[INFO] using runlist: '/cr/data01/filip/xy-calibration/config/calib_runlists/calib_runs_2023-10.list'
[INFO] getting files from '/cr/data02/AugerPrime/XY/2023oct'
[INFO] analyzing 6018m25
[INFO] interpreted as CalA run!
[INFO] reading: 'cal_m25_006018_2023-10-13.01-08.root'
[INFO] LS=nan r=0cm s=0cm C=0.500Gphotons log=none
[INFO] compiling... finished
[INFO] running analysis... finished
[INFO] analyzing 6019
[INFO] interpreted as CalA run!
[INFO] reading: 'cal_m25_006019_2023-10-13.01-25.root'
[INFO] LS=nan r=0cm s=0cm C=0.500Gphotons log=none
[INFO] compiling... finished
[INFO] running analysis... finished
[I

In [ ]:
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt

fIn = "/cr/data01/filip/xy-calibration/campaign_summary/calib_runs_2024-11_OLO_results.csv"

df = pd.read_csv(fIn, comment='#', index_col=0,
                     dtype=defaultdict(lambda: str, global_pixel=int, mirror=int, 
                                       local_pixel=int, col=int, row=int))
df.columns = df.columns.str.strip()  # remove space in keys
label = fIn
label = label.replace("calib_runs_", "").replace("_results", "")

# because there may be nans in the cal_xy col
# pandas parses it as a default string, need to convert
df["cal_xy"] = df['cal_xy'].str.strip()
df["cal_xy"] = df["cal_xy"].astype(float)
df["mirror"] = df["mirror"].astype(float)

dfTelMean = df.groupby(["telescope"]).mean(numeric_only=True).reset_index()
plt.plot(dfTelMean["mirror"], dfTelMean["cal_xy"], '.', label=label)

In [ ]:
df.dtypes

In [ ]:
dfTelMean

In [ ]:
import pandas as pd
from collections import defaultdict

def load_runlist(year_month: str) -> pd.DataFrame:
    data = pd.read_csv(
        f"/cr/data01/filip/xy-calibration/config/calib_runlists/calib_runs_{year_month}.list",
        names=["id","tel","step","date","source","mA","forDB","jobfile","comment"],
        dtype=defaultdict(lambda: str, step="int", mA="float", forDB="bool"),
        index_col=0,
        comment="#",
        sep=";",
    )

    for col in ["source", "jobfile", "comment"]:
        data[col] = data[col].map(lambda x: x.strip().replace('"',''))

    return data

def get_xy_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        runlist['forDB']
        ]

def get_good_xy_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        runlist['forDB']
        & (runlist['step'] == 6)
        & (runlist['source'] == "OLO")
        & (runlist['mA'] == 15.9)
        & (runlist['comment'] == "")
        ]

def get_cal_a_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        (runlist['step'] == 0)
        & (runlist['source'] == "")
        & (runlist['mA'] == 0)
        ]

def get_cal_a_open_shutter_runs(runlist: pd.DataFrame) -> pd.DataFrame:
    return runlist[
        (runlist['step'] == 0)
        & (runlist['source'] == "")
        & (runlist['mA'] == 0)
        & (runlist['comment'].str.contains("open shutter"))
        ]

runlist=load_runlist('2024-11')

get_xy_runs(runlist)

In [ ]:
print(runlist['source'] == "OLO")

In [ ]:
runlist.iloc[0]['source']

In [ ]:
from utils.Auger.SD.Simulation import Simulation

In [ ]:
sim = Simulation(name='test_sim', offline='ds_forced_station_triggers', 
                 src='/cr/users/filip/Projects/SSDTriggerEfficiency/SdSimulationReconstructionUpgrade/',
                 PRIMARY='photon')

In [ ]:
# sim.run(191)

In [ ]:
sim.status()

In [ ]:
sim.process()

In [ ]:
sim.status()